# Human Activity Recognition with Smartphones
## Experiment Plan and Approaches
### 1. Summary of Professor's Advice
- General Obserbation:
    - Typically, activities such as walking, running, sitting, and lying down yield high classification accuracy. However, my results show the opposite - ambiguous activities like walking downstairs/upstairs are classified with higher accuracy. This raises some questions.
- Professor's Recommendations:
    - Preprocessing:
        - Apply different preprocessing methods because different preprocessing approaches yield different results.
    - Compare Different ML Pipelines:
        - Compare various machine learning flows (e.g. LGBM, LSTM, Transfer Learning) to evaluate their performance.
    - Cross Validation Strategy:
        - Use k-fold and Leave-One-Subject-Out (LOSO) cross-validation methods.
    - Transfer Learning:
        - Explore transfer learning approaches.
    - Raw Data Analysis:
        - Try to perform analysis on raw data (i.e., without dimensionality reduction methods like PCA or t-SNE)

### 2. Experiment Plan
Our experiments will be divided into two major categories:

### [1] Data Preprocessing Comparison

### 1. t-SNE Approach:
- Objective: Compare the performance after applying t-SNE for dimensionality reduction followed by LightGBM classification, both in standard splits and LOSO cross-validation.
- Experiments:
    - t-SNE: LightGBM classifier evaluation.
    - t-SNE: LOSO cross validation evaluation.

### 2. Raw Data Approach:
    1. Raw Data Approach:
    - Objective: Compare the performance of a LightGBM classifier using raw data under standard train/test splits versus LOSO cross-validation.
    - Experiments:
        - Raw data -> LightGBM classifier evaluation.
        - Raw data -> LOSO cross-validation evaluation.

---

### [2] Raw Data Analysis (Without Feature Engineering such as PCA/t-SNE)

### 1. Activity Classification Accuracy:
    - Target: Classify activities (e.g, lying, walking, running) using the entire dataset.
    - Methods:
        - Compare approaches using LGBM, LSTM, and Transfer Learning.

### 2. Activity Classification with LOSO:
    - Target: Evaluate activity classification using LOSO cross-validation.
    - Methods:
        - Compare LGBM, LSTM, and Transfer Learning approaches.

### 3. Per-Activity Accuracy:
    - Target: Evaluate the subject classification accuracy for each activity (e.g., LAYING, WALKING, etc.).
    - Methods:
        - Compare LGBM, LSTM, and Transfer Learning results for each activity.

### 4. Per-Activity Accuracy with LOSO:
    - Target: Evaluate the subject classification accuracy for each activity using LOSO cross-validation.
    - Methods:
        - Compare LGBM, LSTM, and Transfer Learning approaches for each activity under LOSO


The key difference between training with a standard train/test split and using LOSO is whether the model has indirectly seen data related to the test set during training. 

With a conventional train/test split, the training data and test data are drawn from the same overall distribution, so the model benefits from patterns that are common across the dataset. 

This typically results in higher accuracy when evaluating on the test set.

---

In contrast, LOSO (Leave-One-Subject-Out) ensures that the model is trained without any exposure to one entire subject’s data. 

This means the model is tested on completely unseen data from a subject it has never encountered before. 

Although this often results in lower accuracy compared to a standard train/test split, LOSO provides a more realistic simulation of how the model would perform when new user data is introduced.

In [54]:
# To store data
import pandas as pd

# To do linear algebra
import numpy as np
from numpy import pi

# To create plots
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import matplotlib

# To create nicer plots
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To get new datatypes and functions
from collections import Counter
from cycler import cycler

# To investigate distributions
from scipy.stats import norm, skew, probplot
from scipy.optimize import curve_fit

# To build models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import LeaveOneGroupOut, KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# To gbm light
from lightgbm import LGBMClassifier

# To measure time
from time import time

# To ignore warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # FutureWarning만 무시
# 혹은 모든 경고 무시: warnings.filterwarnings("ignore")


In [14]:
# check gpu availability
device = 'gpu' if torch.cuda.is_available() else 'cpu'
device

'gpu'

During preprocessing or EDA, it is acceptable to combine the train and test sets to examine the data. 

However, when it comes to model training and final performance evaluation, the train and test sets must be clearly separated to prevent data leakage.

In [82]:
# load dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# combine both datasets
train_df['Data'] = 'Train'
test_df['Data'] = 'Test'

both_df = pd.concat([train_df, test_df], axis = 0).reset_index(drop = True)
both_df['subject'] = '#' + both_df['subject'].astype(str)

label = both_df.pop('Activity')
label_counts = label.value_counts()

print('Shape Train:\t{}'.format(train_df.shape))
print('Shape Test:\t{}'.format(test_df.shape))

train_df.head()

Shape Train:	(7352, 564)
Shape Test:	(2947, 564)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,Data
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING,Train
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING,Train
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING,Train
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING,Train
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING,Train


no null value

In [6]:
print('Null Values in DataFrames: {}\n'.format(both_df.isna().sum().sum()))
both_df.info()

Null Values in DataFrames: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10299 entries, 0 to 10298
Columns: 563 entries, tBodyAcc-mean()-X to Data
dtypes: float64(561), object(2)
memory usage: 44.2+ MB


In [7]:
both_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Data
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,#1,Train
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,#1,Train
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,#1,Train
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,#1,Train
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,#1,Train


In [8]:
raw_data = both_df.copy()
data_data = raw_data.pop('Data')
subject_data = raw_data.pop('subject')

scl = StandardScaler()
raw_data = scl.fit_transform(raw_data)

The reason for using LOSO (Leave-One-Subject-Out) is to verify that the model can generalize well to bew subjects.

We evaluate the data after appluing only scaling without performing dimensionality reduction (e.g., PCA or t-SNE)

[BASE]
- With PCA/t-SNE applied [LGBM]: 0.86%
- With raw data [LGBM]: 0.99%


[LOSO]
- With PCA/t-SNE applied [LGBM]: 0.82% (cross validation reached up to 90% in some folds)
- With raw data [LGBM]: 0.94%

Overall, the raw data performs better.

# Activity accuracy - With raw data [LGBM]

In [16]:
enc = LabelEncoder()
label_encoded = enc.fit_transform(label)
X_train, X_test, y_train, y_test = train_test_split(raw_data, label_encoded, random_state = 3)

# create the model
lgbm = LGBMClassifier(n_estimators = 500, random_state = 3, device = device, verbose = -1)
lgbm = lgbm.fit(X_train, y_train)

score = accuracy_score(y_true = y_test, y_pred = lgbm.predict(X_test))
print('Accuracy on testset:\t{:.4f}'.format(score))

Accuracy on testset:	0.9930


# Activity accuracy - With raw data [LSTM - BASE]

Config model parameter

In [72]:
num_epochs = 50
batch_size = 64
early_stopping_patience = 5

In [55]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, num_classes) # Finally fully connected layer for classification

    def forward(self, x):
        out, (hn, cn) = self.lstm(x) # Use the hidden state from the last time step of each sequence. 
        ## hn and cn represent the cell state, which are often not used in subsequent operations.
        out = out[:, -1, :] # Select the hidden state from the last time step.
        out = self.fc(out) # Pass the last time step's hidden state through the fully connected layer.
        return out

In [67]:
X = raw_data.reshape(raw_data.shape[0], 1, raw_data.shape[1])
y = label_encoded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)

X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train, dtype = torch.long)
y_test_tensor = torch.tensor(y_test, dtype = torch.long)

train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [69]:
model = LSTMClassifier(input_dim = X_train_tensor.shape[2], hidden_dim = 64, num_layers = 1, num_classes = len(np.unique(label_encoded)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels.data)

    epoch_loss = train_loss / len(train_dataset)
    epoch_acc = train_corrects.double() / len(train_dataset)
    print(f'Epoch {epoch + 1}/{num_epochs} Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.4f}')

model.eval()
test_loss = 0.0
test_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects.double() / len(test_dataset)
print(f'\n LSTM Test Accuracy: {test_acc:.4f}')

Epoch 1/50 Loss: 0.7564 Acc: 0.8287
Epoch 2/50 Loss: 0.2484 Acc: 0.9430
Epoch 3/50 Loss: 0.1352 Acc: 0.9649
Epoch 4/50 Loss: 0.0921 Acc: 0.9757
Epoch 5/50 Loss: 0.0750 Acc: 0.9783
Epoch 6/50 Loss: 0.0586 Acc: 0.9824
Epoch 7/50 Loss: 0.0494 Acc: 0.9839
Epoch 8/50 Loss: 0.0406 Acc: 0.9886
Epoch 9/50 Loss: 0.0367 Acc: 0.9881
Epoch 10/50 Loss: 0.0327 Acc: 0.9908
Epoch 11/50 Loss: 0.0264 Acc: 0.9931
Epoch 12/50 Loss: 0.0226 Acc: 0.9943
Epoch 13/50 Loss: 0.0225 Acc: 0.9934
Epoch 14/50 Loss: 0.0191 Acc: 0.9943
Epoch 15/50 Loss: 0.0148 Acc: 0.9970
Epoch 16/50 Loss: 0.0129 Acc: 0.9973
Epoch 17/50 Loss: 0.0114 Acc: 0.9984
Epoch 18/50 Loss: 0.0092 Acc: 0.9988
Epoch 19/50 Loss: 0.0096 Acc: 0.9983
Epoch 20/50 Loss: 0.0068 Acc: 0.9993
Epoch 21/50 Loss: 0.0063 Acc: 0.9992
Epoch 22/50 Loss: 0.0053 Acc: 0.9995
Epoch 23/50 Loss: 0.0043 Acc: 1.0000
Epoch 24/50 Loss: 0.0040 Acc: 0.9999
Epoch 25/50 Loss: 0.0034 Acc: 1.0000
Epoch 26/50 Loss: 0.0033 Acc: 1.0000
Epoch 27/50 Loss: 0.0027 Acc: 1.0000
Epoch 28/5

# Activity accuracy - With raw data [LSTM - early stopping / reduceLR]

In [74]:
model = LSTMClassifier(input_dim = X_train_tensor.shape[2], hidden_dim = 64, num_layers = 1, num_classes = len(np.unique(label_encoded)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

# Use ReduceLROnPlateau to reduce the learning rate when the validation loss stops improving.
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 3, factor = 0.5, verbose = True)

# Initialize early stopping variables
best_val_loss = float('inf')
no_improve_epochs = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_corrects = 0

    # Training phase
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate loss and number of correct predictions.
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_corrects += torch.sum(preds == labels.data)

    epoch_train_loss = train_loss / len(train_dataset)
    epoch_train_acc  = train_corrects.double() / len(train_dataset)
    
    # Validation phase (using test set as validation)
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
    epoch_val_loss = val_loss / len(test_dataset)
    epoch_val_acc  = val_corrects.double() / len(test_dataset)

    # Update the scheduler with the validation loss
    scheduler.step(epoch_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs} '
          f'Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f} || '
          f'Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}')

    # Early stopping check: if validation loss does not improve, increment counter
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= early_stopping_patience:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

# Final evaluateion on the test set
model.eval()
test_loss = 0.0
test_corrects = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)

test_loss = test_loss / len(test_dataset)
test_acc = test_corrects.double() / len(test_dataset)
print(f'\nLSTM Test Accuracy: {test_acc:.4f}')

Epoch 1/50 Train Loss: 0.7298 Acc: 0.8179 || Val Loss: 0.3396 Acc: 0.9311
Epoch 2/50 Train Loss: 0.2365 Acc: 0.9455 || Val Loss: 0.1672 Acc: 0.9563
Epoch 3/50 Train Loss: 0.1312 Acc: 0.9667 || Val Loss: 0.1228 Acc: 0.9646
Epoch 4/50 Train Loss: 0.0942 Acc: 0.9734 || Val Loss: 0.0930 Acc: 0.9733
Epoch 5/50 Train Loss: 0.0703 Acc: 0.9805 || Val Loss: 0.0797 Acc: 0.9757
Epoch 6/50 Train Loss: 0.0606 Acc: 0.9829 || Val Loss: 0.0790 Acc: 0.9762
Epoch 7/50 Train Loss: 0.0494 Acc: 0.9856 || Val Loss: 0.0683 Acc: 0.9767
Epoch 8/50 Train Loss: 0.0460 Acc: 0.9851 || Val Loss: 0.0609 Acc: 0.9782
Epoch 9/50 Train Loss: 0.0402 Acc: 0.9874 || Val Loss: 0.0634 Acc: 0.9762
Epoch 10/50 Train Loss: 0.0328 Acc: 0.9903 || Val Loss: 0.0541 Acc: 0.9796
Epoch 11/50 Train Loss: 0.0320 Acc: 0.9904 || Val Loss: 0.0542 Acc: 0.9796
Epoch 12/50 Train Loss: 0.0240 Acc: 0.9939 || Val Loss: 0.0510 Acc: 0.9782
Epoch 13/50 Train Loss: 0.0222 Acc: 0.9936 || Val Loss: 0.0496 Acc: 0.9801
Epoch 14/50 Train Loss: 0.0194 Acc

# Activity accuracy - With raw data [Transfer Learning]

In [ ]:
## I was unable to find a suitable transfer learning model.

# Raw DATA LOSO

# Activity each subject accuracy [LOSO]- With raw data [LGBM]

In [17]:
logo = LeaveOneGroupOut()

accuracy_list = []
classification_reports = []

for train_idx, test_idx in logo.split(raw_data, label_encoded, subject_data):
    X_train, X_test = raw_data[train_idx], raw_data[test_idx]
    y_train, y_test = label_encoded[train_idx], label_encoded[test_idx]
    
    model = LGBMClassifier(n_estimators = 500, random_state = 3, device = device, verbose =-1)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict = True)

    accuracy_list.append(accuracy)
    classification_reports.append(report)

average_accuracy = sum(accuracy_list) / len(accuracy_list)

print('Accuracy on testset:\t{:.4f}'.format(average_accuracy))


Accuracy on testset:	0.9470


# Activity each subject accuracy [LOSO]- With raw data [LSTM]

In [76]:
accuracy_loso_list = []
classification_loso_reports = []

# Training using LOSO cross-validation over subjects
for fold, (train_idx, test_idx) in enumerate(logo.split(raw_data, label_encoded, subject_data)):
    print(f"\n===== Fold {fold+1} =====")
    # 데이터 분할
    X_train, X_test = raw_data[train_idx], raw_data[test_idx]
    y_train, y_test = label_encoded[train_idx], label_encoded[test_idx]

    # Reshpae the input data 3D for LSTM: (num_samples, sequence_length, num_features)
    # Here, we assume a sequence length of 1.
    X_train_3D = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_3D = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Convert numpy arrays to torch tensors and transfer them to the device
    X_train_tensor = torch.tensor(X_train_3D, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_3D, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    # Create the model
    num_classes = len(np.unique(label_encoded))
    input_dim = X_train.shape[1] # featuer 수(컬럼)
    hidden_dim = 32   # Set desired hidden dimension (the original code used 64 for some experiments)
    num_layers = 1    # Number of LSTM layers (can be modified; currently using a singlt layer)
    model = LSTMClassifier(input_dim, hidden_dim, num_layers, num_classes).to(device)

    # Set the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Create a DataLoader for the training set
    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Epoch traing loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad() # Reset gradients to zero for each batch
            outputs = model(batch_X)  # Output shape: (batch, num_classes)
            loss = criterion(outputs, batch_y)
            loss.backward() # Backpropagate the loss
            optimizer.step()# Update model parameters using the computed gradients
            epoch_loss += loss.item() * batch_X.size(0) # Multiply by batch size to accumulate total loss
        
        # Calculate average loss for the epoch
        epoch_loss = epoch_loss / len(train_loader.dataset)
        
        # Validation phase (using the test set as validation)
        model.eval() # Switch to evaluation mode (this prevents data leakage)
        with torch.no_grad():
            val_outputs = model(X_test_tensor)
            val_loss = criterion(val_outputs, y_test_tensor).item()

        # Verbose output: print the training and validation loss for each epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Evaluation: After training, predict on the test set
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        # Select the class with the highest probability
        _, y_pred_tensor = torch.max(outputs, dim=1)
        y_pred = y_pred_tensor.cpu().numpy() # Convert the GPU tensor to a CPU numpy array

    # Compute the accuracy and classification report
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy_loso_list.append(acc)
    classification_loso_reports.append(report)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

# Calculate the average accuracy across all folds
average_accuracy_loso = np.mean(accuracy_loso_list)
print('\nLOSO LSTM Average Accuracy : {:.4f}'.format(average_accuracy_loso))



===== Fold 1 =====
Epoch 1/50, Train Loss: 0.9427, Val Loss: 0.4726
Epoch 2/50, Train Loss: 0.3448, Val Loss: 0.1931
Epoch 3/50, Train Loss: 0.1887, Val Loss: 0.1036
Epoch 4/50, Train Loss: 0.1276, Val Loss: 0.0806
Epoch 5/50, Train Loss: 0.0988, Val Loss: 0.0634
Epoch 6/50, Train Loss: 0.0802, Val Loss: 0.0434
Epoch 7/50, Train Loss: 0.0652, Val Loss: 0.0391
Epoch 8/50, Train Loss: 0.0575, Val Loss: 0.0384
Epoch 9/50, Train Loss: 0.0495, Val Loss: 0.0342
Epoch 10/50, Train Loss: 0.0423, Val Loss: 0.0306
Epoch 11/50, Train Loss: 0.0366, Val Loss: 0.0425
Epoch 12/50, Train Loss: 0.0343, Val Loss: 0.0248
Epoch 13/50, Train Loss: 0.0297, Val Loss: 0.0258
Epoch 14/50, Train Loss: 0.0265, Val Loss: 0.0222
Epoch 15/50, Train Loss: 0.0255, Val Loss: 0.0233
Epoch 16/50, Train Loss: 0.0224, Val Loss: 0.0218
Epoch 17/50, Train Loss: 0.0213, Val Loss: 0.0225
Epoch 18/50, Train Loss: 0.0175, Val Loss: 0.0165
Epoch 19/50, Train Loss: 0.0160, Val Loss: 0.0143
Epoch 20/50, Train Loss: 0.0141, Val Lo

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch 1/50, Train Loss: 0.9581, Val Loss: 0.5190
Epoch 2/50, Train Loss: 0.3479, Val Loss: 0.2043
Epoch 3/50, Train Loss: 0.1895, Val Loss: 0.1083
Epoch 4/50, Train Loss: 0.1290, Val Loss: 0.0781
Epoch 5/50, Train Loss: 0.0975, Val Loss: 0.0541
Epoch 6/50, Train Loss: 0.0771, Val Loss: 0.0415
Epoch 7/50, Train Loss: 0.0661, Val Loss: 0.0357
Epoch 8/50, Train Loss: 0.0577, Val Loss: 0.0306
Epoch 9/50, Train Loss: 0.0489, Val Loss: 0.0236
Epoch 10/50, Train Loss: 0.0437, Val Loss: 0.0216
Epoch 11/50, Train Loss: 0.0387, Val Loss: 0.0227
Epoch 12/50, Train Loss: 0.0336, Val Loss: 0.0196
Epoch 13/50, Train Loss: 0.0320, Val Loss: 0.0230
Epoch 14/50, Train Loss: 0.0275, Val Loss: 0.0213
Epoch 15/50, Train Loss: 0.0249, Val Loss: 0.0158
Epoch 16/50, Train Loss: 0.0224, Val Loss: 0.0264
Epoch 17/50, Train Loss: 0.0214, Val Loss: 0.0180
Epoch 18/50, Train Loss: 0.0206, Val Loss: 0.0216
Epoch 19/50, Train Loss: 0.0169, Val Loss: 0.0190
Epoch 20/50, Train Loss: 0.0157, Val Loss: 0.0178
Epoch 21/

### [LOSO]- With raw data [LSTM] - ACC: 0.9520

# Activity each subject accuracy [LOSO]- With raw data [LSTM- earlystopping & reduceLR]

In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [60]:
accuracy_loso_list = []
classification_loso_reports = []

for fold, (train_idx, test_idx) in enumerate(logo.split(raw_data, label_encoded, subject_data)):
    print(f"\n===== Fold {fold+1} =====")

    X_train, X_test = raw_data[train_idx], raw_data[test_idx]
    y_train, y_test = label_encoded[train_idx], label_encoded[test_idx]

    X_train_3D = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_3D = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    X_train_tensor = torch.tensor(X_train_3D, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_3D, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    # 모델 생성
    num_classes = len(np.unique(label_encoded))
    input_dim = X_train.shape[1] # featuer 수(컬럼)
    hidden_dim = 64  
    num_layers = 1   
    model = LSTMClassifier(input_dim, hidden_dim, num_layers, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    best_val_loss = float('inf')
    no_improve_epochs = 0

    # 에포크 별 학습 루프
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad() 
            outputs = model(batch_X)  
            loss = criterion(outputs, batch_y)
            loss.backward() 
            optimizer.step()
            epoch_loss += loss.item() * batch_X.size(0) 
        
        epoch_loss = epoch_loss / len(train_loader.dataset)
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test_tensor)
            val_loss = criterion(val_outputs, y_test_tensor).item()

        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improve_epochs = 0
        else:
            no_improve_epochs += 1
        
        if no_improve_epochs >= early_stopping_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        # 가장 높은 확률을 가진 클래스 선택
        _, y_pred_tensor = torch.max(outputs, dim=1)
        y_pred = y_pred_tensor.cpu().numpy()

    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy_loso_list.append(acc)
    classification_loso_reports.append(report)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

average_accuracy_loso = np.mean(accuracy_loso_list)
print('\nLOSO LSTM Average Accuracy : {:.4f}'.format(average_accuracy_loso))



===== Fold 1 =====
Epoch 1/50, Train Loss: 0.3544, Val Loss: 0.1117
Epoch 2/50, Train Loss: 0.0915, Val Loss: 0.0428
Epoch 3/50, Train Loss: 0.0663, Val Loss: 0.0303
Epoch 4/50, Train Loss: 0.0510, Val Loss: 0.0230
Epoch 5/50, Train Loss: 0.0465, Val Loss: 0.0230
Epoch 6/50, Train Loss: 0.0373, Val Loss: 0.0290
Epoch 7/50, Train Loss: 0.0371, Val Loss: 0.0338
Epoch 00008: reducing learning rate of group 0 to 5.0000e-04.
Epoch 8/50, Train Loss: 0.0290, Val Loss: 0.0496
Epoch 9/50, Train Loss: 0.0212, Val Loss: 0.0133
Epoch 10/50, Train Loss: 0.0158, Val Loss: 0.0120
Epoch 11/50, Train Loss: 0.0145, Val Loss: 0.0149
Epoch 12/50, Train Loss: 0.0130, Val Loss: 0.0176
Epoch 13/50, Train Loss: 0.0107, Val Loss: 0.0164
Epoch 00014: reducing learning rate of group 0 to 2.5000e-04.
Epoch 14/50, Train Loss: 0.0088, Val Loss: 0.0123
Epoch 15/50, Train Loss: 0.0062, Val Loss: 0.0109
Epoch 16/50, Train Loss: 0.0054, Val Loss: 0.0115
Epoch 17/50, Train Loss: 0.0050, Val Loss: 0.0119
Epoch 18/50, Tr

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\vivid\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Epoch 1/50, Train Loss: 0.3587, Val Loss: 0.0995
Epoch 2/50, Train Loss: 0.0918, Val Loss: 0.0447
Epoch 3/50, Train Loss: 0.0641, Val Loss: 0.0264
Epoch 4/50, Train Loss: 0.0477, Val Loss: 0.0281
Epoch 5/50, Train Loss: 0.0455, Val Loss: 0.0102
Epoch 6/50, Train Loss: 0.0355, Val Loss: 0.0166
Epoch 7/50, Train Loss: 0.0313, Val Loss: 0.0200
Epoch 8/50, Train Loss: 0.0272, Val Loss: 0.0137
Epoch 00009: reducing learning rate of group 0 to 5.0000e-04.
Epoch 9/50, Train Loss: 0.0240, Val Loss: 0.0144
Epoch 10/50, Train Loss: 0.0153, Val Loss: 0.0134
Early stopping triggered at epoch 10
Fold 7 Accuracy: 0.9970

===== Fold 8 =====
Epoch 1/50, Train Loss: 0.3428, Val Loss: 0.3305
Epoch 2/50, Train Loss: 0.0830, Val Loss: 0.3130
Epoch 3/50, Train Loss: 0.0609, Val Loss: 0.2666
Epoch 4/50, Train Loss: 0.0467, Val Loss: 0.2407
Epoch 5/50, Train Loss: 0.0368, Val Loss: 0.4132
Epoch 6/50, Train Loss: 0.0291, Val Loss: 0.2513
Epoch 7/50, Train Loss: 0.0258, Val Loss: 0.2694
Epoch 00008: reducing l

### [LOSO]- With raw data [LSTM- earlystopping & reduceLR] - ACC: 0.9491

# [Each Activity Accuracy] - With raw data [LGBM] 

In [39]:
label_counts = label.value_counts()
lgbm_raw_data = []

for activity in label_counts.index:
    act_data = both_df[label == activity].copy()
    act_data_data = act_data.pop('Data')
    act_subject_data = act_data.pop('subject')

    # scale the data
    scl = StandardScaler()
    act_data = scl.fit_transform(act_data)

    enc = LabelEncoder()
    label_encoded = enc.fit_transform(act_subject_data)
    X_train, X_test, y_train, y_test = train_test_split(act_data, label_encoded, random_state = 3)

    print('Activity: {}'.format(activity))
    lgbm = LGBMClassifier(n_estimators = 500, random_state = 3, verbose =-1)
    lgm = lgbm.fit(X_train, y_train)

    score = accuracy_score(y_true = y_test, y_pred = lgbm.predict(X_test))
    print('Accuracy on testset\t{:.4f}\n'.format(score))
    lgbm_raw_data.append([activity, score])

Activity: LAYING
Accuracy on testset	0.9198

Activity: STANDING
Accuracy on testset	0.8721

Activity: SITTING
Accuracy on testset	0.7618

Activity: WALKING
Accuracy on testset	0.9861

Activity: WALKING_UPSTAIRS
Accuracy on testset	0.9819

Activity: WALKING_DOWNSTAIRS
Accuracy on testset	0.9830



# [Each Activity Accuracy] - With raw data [LSTM] 

In [92]:
# activity별로 정확도 lstm
lstm_raw_data = []
for activity in label_counts.index:
    act_data = both_df[label == activity] #label value와 activiy가 같은 데이터 index 출력
    act_data_data = act_data.pop('Data')
    act_subject_data = act_data.pop('subject')

    scl = StandardScaler()
    act_data_scaled = scl.fit_transform(act_data)

    enc = LabelEncoder()
    label_encoded = enc.fit_transform(act_subject_data)

    X_train, X_test, y_train, y_test = train_test_split(act_data_scaled, label_encoded, random_state = 3) 

    print(f'Activity: {activity}')

    X_train_3D = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_3D = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    X_train_tensor = torch.tensor(X_train_3D, dtype = torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_3D, dtype = torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype = torch.long).to(device)
    y_test_tensor = torch.tensor(y_test, dtype = torch.long).to(device)

    input_dim = X_train_tensor.shape[2] # feature 수
    num_classes = len(np.unique(label_encoded))
    model = LSTMClassifier(input_dim, hidden_dim, num_layers, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)

    train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
        epoch_loss = epoch_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1} / {num_epochs}, Train Loss : {epoch_loss:.4f}')
    
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, y_pred_tensor = torch.max(outputs, dim = 1)
        y_pred = y_pred_tensor.cpu().numpy()

    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy on testset for activity {activity}: {acc:.4f}\n')

    lstm_raw_data.append([activity, acc])

print('Final LSTM Results per Activity:')
for activity, score in lstm_raw_data:
    print(f'Activity: {activity}, Accuracy: {score:.4f}')

Activity: LAYING
Epoch 1 / 50, Train Loss : 3.4290
Epoch 2 / 50, Train Loss : 3.2212
Epoch 3 / 50, Train Loss : 3.0527
Epoch 4 / 50, Train Loss : 2.8620
Epoch 5 / 50, Train Loss : 2.6674
Epoch 6 / 50, Train Loss : 2.4801
Epoch 7 / 50, Train Loss : 2.3020
Epoch 8 / 50, Train Loss : 2.1376
Epoch 9 / 50, Train Loss : 1.9863
Epoch 10 / 50, Train Loss : 1.8482
Epoch 11 / 50, Train Loss : 1.7236
Epoch 12 / 50, Train Loss : 1.6088
Epoch 13 / 50, Train Loss : 1.5023
Epoch 14 / 50, Train Loss : 1.4073
Epoch 15 / 50, Train Loss : 1.3177
Epoch 16 / 50, Train Loss : 1.2398
Epoch 17 / 50, Train Loss : 1.1648
Epoch 18 / 50, Train Loss : 1.0931
Epoch 19 / 50, Train Loss : 1.0310
Epoch 20 / 50, Train Loss : 0.9712
Epoch 21 / 50, Train Loss : 0.9159
Epoch 22 / 50, Train Loss : 0.8646
Epoch 23 / 50, Train Loss : 0.8169
Epoch 24 / 50, Train Loss : 0.7719
Epoch 25 / 50, Train Loss : 0.7295
Epoch 26 / 50, Train Loss : 0.6888
Epoch 27 / 50, Train Loss : 0.6526
Epoch 28 / 50, Train Loss : 0.6189
Epoch 29 / 5

Limited Data for LSTM Training:

It appears that the dataset is not large enough for the LSTM model to learn effectively. With limited data, the LSTM may struggle to capture complex temporal patterns, leading to poor generalization and suboptimal performance.

# [Each Activity Accuracy [LOSO] ] - With raw data [LGBM] 

In [117]:
lgbm_raw_data  = []

for activity in label_counts.index:
    if activity != 'WALKING':
        continue
    
    act_data = both_df[label == activity].copy()
    act_data_data = act_data.pop('Data')
    act_subject_data = act_data.pop('subject')

    # StandardScaler 적용
    scl = StandardScaler()
    act_data_scaled = scl.fit_transform(act_data)

    # Label Encoding 적용
    enc = LabelEncoder()
    label_encoded = enc.fit_transform(act_subject_data)

    groups = act_subject_data.values
    
    logo = LeaveOneGroupOut()

    fold_scores = []

    # LOSO 적용
    for train_idx, test_idx in logo.split(act_data_scaled, label_encoded, groups = groups):
        X_train, X_test = act_data_scaled[train_idx], act_data_scaled[test_idx]
        y_train, y_test = label_encoded[train_idx], label_encoded[test_idx]

        # 모델 학습
        lgbm = LGBMClassifier(n_estimators = 500, random_state = 3, device = 'gpu', verbose = -1) # LGBM GPU사용은 직접 지정
        lgbm.fit(X_train, y_train)

        # 예측 및 평가
        score = accuracy_score(y_true = y_test, y_pred = lgbm.predict(X_test))
        fold_scores.append(score)
    
    avg_score = np.mean(fold_scores)
    print(f"Activity: {activity}")
    print(f'LOSO Accuracy: {avg_score:.4f}\n')

    lgbm_raw_data.append([activity, avg_score])

print('Final LGBM Results per Activity (LOSO):')
for activity, score in lgbm_raw_data:
    print(f"Activity: {activity}, Accuracy: {score:.4f}")

Activity: WALKING
LOSO Accuracy: 0.0000

Final LGBM Results per Activity (LOSO):
Activity: WALKING, Accuracy: 0.0000


# ???

LOSO Performance Issues for Each Activity:

When applying Leave-One-Subject-Out (LOSO) cross-validation for each activity, the model completely fails to learn—for example, the accuracy for the "LAYING" activity is 0. 

I initially suspected that "LAYING" data might be inherently challenging to learn. 

However, even when I conducted experiments on the "WALKING" activity using a non-LOSO approach with an LGBM model (which achieved 99% accuracy), the model still failed to learn effectively under the LOSO scheme.